In [ ]:
# -*- coding: utf-8 -*-
"""
SNATCH 프로젝트: E2E Fine-tuning 데이터셋 구축 및 튜닝 시작 (Colab)

1. YouTube 댓글 수집 (API Key 필요)
2. 데이터 전처리 및 필터링
3. LLM(Gemini 1.5 Pro) 기반 초벌 JSON 분석 결과 생성 (API Key 필요, 비용/시간 소요)
4. Fine-tuning용 JSONL 파일 생성
5. Google AI Studio API를 이용한 Fine-tuning 작업 시작 (API Key 필요)

**주의:** 실제 프로젝트에서는 3단계 이후 반드시 사람의 검토/수정 필요!
"""

In [2]:
# === Stage 0: 설정 및 라이브러리 임포트 ===
print("Stage 0: 설정 및 라이브러리 임포트 시작")

# 1. 필요한 라이브러리 설치
!pip install --upgrade google-api-python-client pandas google-auth-oauthlib google-auth-httplib2 langdetect tqdm google-generativeai

Stage 0: 설정 및 라이브러리 임포트 시작
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 21.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 9.3 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=14cfc4fa8ee386348d4ea6271d2c36c01308391828c16aa98decf61677b4371b
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: google-

In [1]:
# 라이브러리 임포트
import os
import pandas as pd
import json
import re
import time
import uuid
from datetime import datetime # datetime 임포트 추가
from google.colab import auth, userdata
from google.colab import files # Colab 파일 다운로드용 (선택 사항)
from google.api_core import retry
from google.api_core import exceptions as core_exceptions
import googleapiclient.discovery
import googleapiclient.errors
from langdetect import detect, LangDetectException
import google.generativeai as genai # google.generativeai로 임포트 이름 변경 (최신 SDK)
from google.generativeai import types
from tqdm.notebook import tqdm
import shutil # 파일 복사 shutil 임포트 추가

print("라이브러리 임포트 완료.")

라이브러리 임포트 완료.


In [2]:
# 3. GCP 및 사용자 설정
try:
    # Colab 사용자 인증
    auth.authenticate_user()
    print("Colab 인증 완료.")

    # --- 사용자 설정 필요한 변수 ---
    # !! Google AI Studio API 키를 Colab Secrets에 'GOOGLE_API_KEY'와 'YOUTUBE_API_KEY'로 저장하세요 !!
    API_KEY = userdata.get('YOUTUBE_API_KEY') # YouTube API 키도 Secrets 사용 권장
    if not API_KEY:
        raise ValueError("YouTube API Key ('YOUTUBE_API_KEY') not found in Colab Secrets.")
    print("YouTube API 키 로드 완료.")

    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    if not GOOGLE_API_KEY:
         raise ValueError("Google AI Studio API Key ('GOOGLE_API_KEY') not found in Colab Secrets.")
    print("Google AI Studio API 키 로드 완료.")

    # Vertex AI 등 다른 GCP 서비스와 연동 시 필요하며, API 동작에 영향을 줄 수 있습니다.
    # 명시적으로 설정해두는 것이 좋습니다.
    PROJECT_ID = "your-gcp-project-id" # 여기에 본인의 GCP Project ID 입력
    LOCATION = "us-central1" # Vertex AI 사용 리전 (예: us-central1)

    # YouTube 검색 설정
    SEARCH_QUERY = "CU 밤티라미수" # 댓글을 수집할 제품명/키워드
    NUM_VIDEOS_TO_SEARCH = 3      # 검색할 관련 영상 최대 개수 (API 쿼터 고려)
    NUM_COMMENTS_PER_VIDEO = 50   # 각 영상에서 수집할 최대 댓글 수 (API 쿼터 고려)

    # 데이터 처리 설정
    RAW_COMMENTS_CSV = f"{SEARCH_QUERY.replace(' ', '_')}_raw_comments.csv"
    FILTERED_COMMENTS_CSV = f"{SEARCH_QUERY.replace(' ', '_')}_filtered_comments.csv"
    PRE_REVIEW_JSONL_PATH = "generated_prereview_data.jsonl" # 검토 전 데이터
    FINAL_TUNING_JSONL_PATH = "final_finetuning_data.jsonl"   # 최종 튜닝 데이터 (여기선 검토 생략)

    # LLM 설정
    GENERATION_MODEL_ID = "gemini-1.5-pro-001" # 초벌 생성용 모델 ID
    TUNING_BASE_MODEL_ID = "gemini-1.5-flash-001-tuning" # Fine-tuning 대상 모델 ID (API 요구사항 확인)
    TUNED_MODEL_DISPLAY_NAME = "snatch_review_analyzer_v1" # AI Studio에 표시될 이름

    # 처리 제한 설정 (비용/시간 관리용)
    MAX_COMMENTS_TO_PROCESS = 10 # 초기 테스트 시 작게 설정! (예: 10)
    API_CALL_DELAY = 1.5         # API 호출 간 지연 시간 (초) - Rate Limiting 방지
    # -----------------------------

    # google-genai SDK 초기화 (configure 대신 다른 방식 사용)
    # API 키는 generate_content 호출 시 model 인자에 포함되거나 환경 변수를 통해 설정됩니다.
    # os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY # 환경 변수 설정 (일부 SDK에서 사용)
    genai.configure(api_key=GOOGLE_API_KEY) # 이전 버전 방식 (오류 원인 추정)

    print("Google AI Studio Client 초기화 준비 완료.")
    settings_ok = True

except Exception as e:
    print(f"설정 및 초기화 중 오류 발생: {e}")
    print("Colab Secrets에 'YOUTUBE_API_KEY'와 'GOOGLE_API_KEY'를 정확히 설정했는지 확인하세요.")
    settings_ok = False

Colab 인증 완료.
YouTube API 키 로드 완료.
Google AI Studio API 키 로드 완료.
Google AI Studio Client 초기화 준비 완료.


In [3]:
# === Stage 1: YouTube 댓글 데이터 수집 ===
# ... (이전 답변에서 제공한 get_youtube_client, search_videos, get_video_comments 함수 그대로 사용)

def get_youtube_client(api_key):
    """YouTube Data API v3 클라이언트를 초기화합니다."""
    try:
        api_service_name = "youtube"
        api_version = "v3"
        # build 함수는 developerKey 인자를 사용하여 API 키를 전달합니다.
        youtube = googleapiclient.discovery.build(
            api_service_name, api_version, developerKey=api_key)
        print("YouTube 클라이언트 초기화 성공.")
        return youtube
    except googleapiclient.errors.HttpError as e:
        print(f"YouTube 클라이언트 초기화 실패 (HttpError): {e}")
        # API 키가 유효하지 않거나 서비스 사용 설정이 안 된 경우 등
        return None
    except Exception as e:
        print(f"YouTube 클라이언트 초기화 중 예상치 못한 오류 발생: {e}")
        return None

def search_videos(youtube, query, max_results):
    """주어진 쿼리로 YouTube 영상을 검색하고 관련 정보를 반환합니다."""
    videos = []
    try:
        request = youtube.search().list(
            part="snippet",
            q=query,
            type="video",
            maxResults=max_results,
            order="relevance", # 관련성 높은 순
            relevanceLanguage="ko", # 한국어 영상 우선 고려 (선택 사항)
            regionCode="KR" # 한국 지역 결과 우선 고려 (선택 사항)
        )
        response = request.execute()

        for item in response.get('items', []):
            if item['id']['kind'] == 'youtube#video':
                videos.append({
                    'video_id': item['id']['videoId'],
                    'video_title': item['snippet']['title'],
                    'channel_title': item['snippet']['channelTitle'],
                    'published_at': item['snippet']['publishedAt']
                })
        print(f"'{query}' 검색 결과: {len(videos)}개의 영상을 찾았습니다 (최대 {max_results}개).")
        return videos
    except googleapiclient.errors.HttpError as e:
        print(f"영상 검색 중 오류 발생 (HttpError): {e}")
        # API 쿼터 초과 등의 문제일 수 있음
        return []
    except Exception as e:
        print(f"영상 검색 중 예상치 못한 오류 발생: {e}")
        return []

def get_video_comments(youtube, video_id, max_results):
    """특정 영상의 댓글을 수집합니다 (최상위 댓글만)."""
    comments = []
    try:
        # 댓글 스레드를 가져오는 요청 (최상위 댓글만)
        request = youtube.commentThreads().list(
            part="snippet",
            videoId=video_id,
            maxResults=min(100, max_results), # API는 한 번에 최대 100개 요청 가능
            order="relevance", # 관련성 또는 최신순(time) 선택 가능
            textFormat="plainText" # HTML 대신 일반 텍스트로 받기
        )
        response = request.execute()

        while response and len(comments) < max_results:
            for item in response.get('items', []):
                comment = item['snippet']['topLevelComment']['snippet']
                comments.append({
                    'comment_id': item['snippet']['topLevelComment']['id'],
                    'comment_text': comment['textDisplay'],
                    'comment_author': comment.get('authorDisplayName', 'Unknown Author'), # 일부 채널은 이름 비공개
                    'comment_published_at': comment['publishedAt'],
                    'like_count': comment.get('likeCount', 0) # 좋아요 수 (선택 사항)
                })
                if len(comments) >= max_results:
                    break

            # 다음 페이지가 있고, 아직 요청한 댓글 수에 도달하지 못했다면 다음 페이지 토큰 사용
            if 'nextPageToken' in response and len(comments) < max_results:
                next_page_token = response['nextPageToken']
                request = youtube.commentThreads().list(
                    part="snippet",
                    videoId=video_id,
                    maxResults=min(100, max_results - len(comments)),
                    order="relevance",
                    textFormat="plainText",
                    pageToken=next_page_token
                )
                response = request.execute()
            else:
                break # 다음 페이지 없거나 목표 수량 도달 시 종료

        print(f"  - 영상 ID '{video_id}': {len(comments)}개의 댓글 수집 완료.")
        return comments

    except googleapiclient.errors.HttpError as e:
        # 댓글 기능이 비활성화된 영상 등의 경우 오류 발생 가능
        error_details = e.resp.get('content-type', ''), e.content
        if 'application/json' in error_details[0]:
            try:
                error_json = json.loads(error_details[1])
                reason = error_json.get('error', {}).get('errors', [{}])[0].get('reason')
                if reason == 'commentsDisabled':
                    print(f"  - 경고: 영상 ID '{video_id}'의 댓글 기능이 비활성화되어 있습니다.")
                elif reason == 'forbidden':
                     print(f"  - 경고: 영상 ID '{video_id}'의 댓글에 접근할 권한이 없습니다 (API 키 문제 또는 비공개 영상 등).")
                else:
                    print(f"  - 오류: 영상 ID '{video_id}' 댓글 수집 중 오류 발생 (HttpError): {reason} / {e}")
            except json.JSONDecodeError:
                 print(f"  - 오류: 영상 ID '{video_id}' 댓글 수집 중 오류 발생 (HttpError, JSON 파싱 불가): {e}")
        else:
             print(f"  - 오류: 영상 ID '{video_id}' 댓글 수집 중 오류 발생 (HttpError): {e}")
        return []
    except Exception as e:
        print(f"  - 오류: 영상 ID '{video_id}' 댓글 수집 중 예상치 못한 오류 발생: {e}")
        return []


def collect_youtube_data(api_key, search_query, num_videos, num_comments_video, output_csv):
    """YouTube 영상 검색 및 댓글 수집 메인 함수"""
    print("\nStage 1: YouTube 댓글 데이터 수집 시작")
    youtube_client = get_youtube_client(api_key)
    if not youtube_client:
        print("YouTube 클라이언트 초기화 실패.")
        print("Stage 1: YouTube 댓글 데이터 수집 완료 (실패).\n")
        return None

    videos_found = search_videos(youtube_client, search_query, num_videos)
    all_comments_data = []

    if videos_found:
        for video in tqdm(videos_found, desc="영상별 댓글 수집 중"):
            video_id = video['video_id']
            video_title = video['video_title']
            print(f"\n영상 처리 중: {video_title} (ID: {video_id})")
            comments = get_video_comments(youtube_client, video_id, num_comments_video)
            for comment in comments:
                comment['video_id'] = video_id
                comment['video_title'] = video_title
            all_comments_data.extend(comments)
        print(f"\n총 {len(all_comments_data)}개의 댓글을 수집했습니다.")
        if all_comments_data:
            df_comments = pd.DataFrame(all_comments_data)
            df_comments = df_comments[['video_id', 'video_title', 'comment_id', 'comment_published_at', 'comment_author', 'comment_text']]
            try:
                df_comments.to_csv(output_csv, index=False, encoding='utf-8-sig')
                print(f"Raw 댓글 데이터를 '{output_csv}'로 저장했습니다.")
                print("Stage 1: YouTube 댓글 데이터 수집 완료.\n")
                return df_comments
            except Exception as e:
                 print(f"Raw 댓글 데이터 저장 중 오류 발생: {e}")
                 print("Stage 1: YouTube 댓글 데이터 수집 완료 (저장 실패).\n")
                 return pd.DataFrame() # 빈 DataFrame 반환
        else:
            print("수집된 댓글 데이터가 없습니다.")
            print("Stage 1: YouTube 댓글 데이터 수집 완료.\n")
            return pd.DataFrame() # 빈 DataFrame 반환
    else:
        print(f"'{search_query}'에 대한 관련 영상을 찾지 못했습니다.")
        print("Stage 1: YouTube 댓글 데이터 수집 완료 (영상 없음).\n")
        return pd.DataFrame()

# === Stage 2: 데이터 전처리 및 필터링 ===
# ... (이전 답변에서 제공한 preprocess_and_filter_comments 함수 그대로 사용)

def preprocess_and_filter_comments(df):
    """댓글 DataFrame을 전처리하고 필터링합니다."""
    print("Stage 2: 데이터 전처리 및 필터링 시작")
    if 'comment_text' not in df.columns:
        print("경고: 'comment_text' 컬럼이 없습니다. 빈 DataFrame을 반환합니다.")
        return pd.DataFrame()

    initial_count = len(df)
    if initial_count == 0:
        print("입력 데이터가 비어있습니다.")
        print("Stage 2: 데이터 전처리 및 필터링 완료 (데이터 없음).\n")
        return df
    print(f"초기 댓글 수 = {initial_count}")

    df_processed = df.copy()
    df_processed['comment_text_clean'] = df_processed['comment_text'].astype(str).str.strip()
    df_processed.dropna(subset=['comment_text_clean'], inplace=True) # NaN 값 제거
    df_processed = df_processed[df_processed['comment_text_clean'] != ''] # 빈 문자열 제거

    min_length = 10
    df_processed = df_processed[df_processed['comment_text_clean'].str.len() >= min_length]
    count_after_len = len(df_processed)
    print(f"  - 길이 필터링 후: {count_after_len} (제외: {initial_count - count_after_len})")
    if df_processed.empty:
        print("Stage 2: 데이터 전처리 및 필터링 완료 (필터링 후 데이터 없음).\n")
        return df_processed

    def is_korean(text):
        try:
            # 작은 텍스트에 대한 언어 감지 오류를 줄이기 위해 최소 길이 확인 (langdetect의 한계)
            if len(text) < 5: return False
            lang = detect(text)
            return lang == 'ko'
        except LangDetectException:
             # print(f"    언어 감지 실패: {text[:30]}...") # 디버깅용
             return False
        except Exception as e:
             print(f"    언어 감지 중 예상치 못한 오류: {e}")
             return False

    # tqdm으로 진행률 표시하며 언어 감지 적용
    korean_mask = [is_korean(text) for text in tqdm(df_processed['comment_text_clean'], desc="  - 언어 감지 중")]
    df_processed = df_processed[korean_mask]
    count_after_lang = len(df_processed)
    print(f"  - 한국어 필터링 후: {count_after_lang} (제외: {count_after_len - count_after_lang})")
    if df_processed.empty:
        print("Stage 2: 데이터 전처리 및 필터링 완료 (한국어 필터링 후 데이터 없음).\n")
        return df_processed

    patterns_to_remove = [
        r'https?://\S+',
        r'구독\s?좋아요|구독과 좋아요|구독 부탁',
        r'^\?+$', r'^\!+$', r'^ㅋ+$', r'^ㅎ+$', r'^\s*$',
        r'어디서\s+(사|팔|구매)|(얼마|가격).*(\?|요$)|재고\s+있',
        r'^(\W|\s)+$'
    ]
    combined_pattern = '|'.join(patterns_to_remove)
    # na=False 추가하여 NaN 값 처리 방지
    noise_mask = ~df_processed['comment_text_clean'].str.contains(combined_pattern, regex=True, na=False)
    df_processed = df_processed[noise_mask]
    final_count = len(df_processed)
    print(f"  - 규칙 기반 필터링 후: {final_count} (제외: {count_after_lang - final_count})")

    print(f"Stage 2: 데이터 전처리 및 필터링 완료. 최종 댓글 수 = {final_count}\n")
    # fine-tuning에 필요한 컬럼과 원본 댓글 텍스트를 포함하여 반환
    return df_processed[['comment_id', 'comment_text']]

# === Stage 3: LLM 기반 초벌 JSON 생성 ===
# ... (이전 답변에서 제공한 create_generation_prompt, generate_structured_output_with_retry 함수 그대로 사용)

def create_generation_prompt(comment_text):
    """LLM에 전달할 상세 프롬프트를 생성합니다."""
    # !! 여기에 최종적으로 합의된 상세 프롬프트 가이드 전체 내용을 붙여넣으세요 !!
    # 예시 프롬프트 (반드시 실제 프로젝트에 맞게 수정/확장 필요)
    prompt = f"""
# 역할:
당신은 시니어 편의점 디저트 VoC 분석 전문가입니다. 주어진 고객 리뷰 텍스트를 분석하여, 정의된 JSON 형식에 따라 상세 분석 결과를 생성합니다.

# 작업 목표:
주어진 '리뷰 텍스트'(`textInput`)를 분석하여 다음 '출력 JSON 형식'(`output`)에 맞는 JSON 데이터를 생성합니다.

# 출력 JSON 형식 (목표):
```json
{{
  "attributes": {{
    "맛": {{ "sentiment": "[positive|negative|neutral|null]", "descriptors": ["[descriptor1", ...]|null] }},
    "식감": {{ "sentiment": "[positive|negative|neutral|null]", "descriptors": ["[descriptor1", ...]|null] }},
    "가격": {{ "sentiment": "[positive|negative|neutral|null]", "descriptors": ["[descriptor1", ...]|null] }},
    "포장": {{ "sentiment": "[positive|negative|neutral|null]", "descriptors": ["[descriptor1", ...]|null] }},
    "재구매": {{ "sentiment": "[positive|negative|neutral|null]", "descriptors": ["[descriptor1", ...]|null] }}
  }},
  "meta": {{
    "날짜언급": "[텍스트에서 추출된 날짜 관련 언급]" | null,
    "편의점언급": "[텍스트에서 추출된 편의점 브랜드]" | null
  }},
  "is_noise": [true|false],
  "overall_sentiment": "[positive|negative|neutral|mixed|null]"
}}

상세 분석 지침:
Noise 판단 (is_noise): 리뷰 텍스트가 제품과 무관하면 true, 아니면 false.
속성별 분석 (attributes): 맛, 식감, 가격, 포장, 재구매 의사에 대한 언급과 감성(positive/negative/neutral), 구체적 표현(descriptors 리스트) 추출. 언급 없으면 null.
메타 정보 추출 (meta): 텍스트 내 날짜/편의점 언급 추출. 없으면 null.
전체 감성 판단 (overall_sentiment): 리뷰 전체의 주된 감성(positive/negative/neutral/mixed) 판단.
예시 데이터:
(여기에 몇 가지 좋은 예시 포함)
예시 1:
리뷰 텍스트: "CU 신상 밤 티라미수 먹어봤는데 크림은 완전 부드럽고 맛있음! 근데 밑에 빵이 좀 퍽퍽하고 가격도 살짝 비싼 느낌 ㅠ 그래도 한번쯤 먹어볼 만해"
출력 JSON: json {{"attributes": {{"맛": {{"sentiment": "positive", "descriptors": ["크림 맛있음"]}}, "식감": {{"sentiment": "mixed", "descriptors": ["크림 완전 부드러움", "빵 퍽퍽함"]}}, "가격": {{"sentiment": "negative", "descriptors": ["살짝 비싼 느낌"]}}, "포장": null, "재구매": {{"sentiment": "neutral", "descriptors": ["한번쯤 먹어볼 만해"]}}}}, "meta": {{"날짜언급": null, "편의점언급": "CU"}}, "is_noise": false, "overall_sentiment": "mixed"}}
생성 요청:
이제 다음 '리뷰 텍스트'를 분석하여 위의 지침과 형식에 맞는 '출력 JSON'을 생성해주세요. 반드시 JSON 형식만 출력하고 다른 부연 설명은 포함하지 마세요.
리뷰 텍스트:
{comment_text}
출력 JSON:
"""
    return prompt

# Google AI Studio API 호출 재시도 설정
@retry.Retry(predicate=retry.if_exception_type(
    core_exceptions.ResourceExhausted, # Rate limiting 오류
    core_exceptions.InternalServerError, # 서버 내부 오류
    core_exceptions.ServiceUnavailable # 서비스 일시 불가
), tries=5, delay=2, backoff=2) # 최대 5번 재시도, 2초부터 시작, 2배 증가
def generate_structured_output_with_retry(model, comment_text):
    """재시도 로직을 포함하여 Gemini 모델을 호출합니다."""
    if not comment_text or len(comment_text.strip()) < 5:
        return None
    prompt = create_generation_prompt(comment_text)
    try:
        # Google AI Studio Gemini 모델 호출 (api_key 직접 전달)
        # model 인자에 "models/gemini-1.5-pro-001" 또는 "gemini-1.5-pro-001" 형태로 전달 (SDK 버전 확인)
        # google-generativeai SDK의 최신 버전은 model 인자에 API 키를 직접 전달하는 방식 지원 안 할 수 있음.
        # 이 경우, genai.configure(api_key=...) 방식이 필요하거나, 환경 변수 설정 필요.
        # 여기서는 환경 변수 설정 방식이 작동한다고 가정합니다.
        model_instance = genai.GenerativeModel(model_name=model.model_name)

        response = model_instance.generate_content(
            prompt,
            generation_config={
                "temperature": 0.1,
                "max_output_tokens": 2048
            }
        )
        # 응답에서 JSON 문자열 추출 및 정리
        generated_text = response.text.strip()
        if generated_text.startswith("```json"):
            generated_text = generated_text[7:]
        if generated_text.endswith("```"):
            generated_text = generated_text[:-3]
        generated_text = generated_text.strip()

        # 유효한 JSON인지 검증
        json.loads(generated_text)
        return generated_text
    except json.JSONDecodeError as e:
        print(f"\n  경고: LLM이 유효하지 않은 JSON 생성. 오류: {e}")
        # print(f"  LLM Raw Response: {response.text[:200] if hasattr(response, 'text') else 'N/A'}...") # 디버깅용
        return None
    except Exception as e: # API 호출 오류 등 기타 예외
        print(f"\n  오류: API 호출 중 예외 발생: {e}")
        raise # 재시도 predicate에 의해 처리되도록 예외 발생 시킴

def generate_llm_outputs(df_filtered, model_id, output_jsonl_path, max_comments, delay, api_key):
    """필터링된 댓글에 대해 LLM으로 초벌 JSON 생성을 수행합니다."""
    print("\nStage 3: LLM 기반 초벌 JSON 생성 시작")
    results_for_jsonl = []
    processed_count = 0
    error_count = 0

    # 처리할 댓글 수 제한
    df_to_process = df_filtered.head(max_comments)
    print(f"총 {len(df_to_process)}개의 댓글에 대해 LLM 분석을 시도합니다 (최대 {max_comments}개).")

    # Google AI Studio Gemini モデル 호출 객체 생성 (API 키는 환경 변수에서 자동 로드 가정)
    # genai.configure(api_key=api_key) # 이전 버전 방식 - 제거
    generation_model = genai.GenerativeModel(model_name=model_id)

    for index, row in tqdm(df_to_process.iterrows(), total=df_to_process.shape[0], desc="LLM 분석 중"):
        comment_text = str(row['comment_text'])

        # 빈 댓글 또는 너무 짧은 댓글 스킵
        if not comment_text or len(comment_text.strip()) < 10:
            error_count += 1
            continue

        # 재시도 포함 LLM 호출
        structured_output_json_str = generate_structured_output_with_retry(generation_model, comment_text)

        if structured_output_json_str:
            results_for_jsonl.append({
                "textInput": comment_text,
                "output": structured_output_json_str
            })
            processed_count += 1
        else:
            error_count += 1

        time.sleep(delay) # API Rate Limiting 방지

    print(f"\nStage 3: LLM 기반 초벌 JSON 생성 완료. 성공 {processed_count}건, 실패 {error_count}건")

    if results_for_jsonl:
        print(f"\n결과를 JSONL 파일로 저장 중: {output_jsonl_path}")
        try:
            with open(output_jsonl_path, 'w', encoding='utf-8') as f:
                for item in results_for_jsonl:
                    f.write(json.dumps(item, ensure_ascii=False) + '\n')
            print("JSONL 파일 저장 완료.")
            # files.download(output_jsonl_path) # 다운로드 링크 제공 (Colab 기본 기능)
            print("\nStage 3: LLM 기반 초벌 JSON 생성 완료.\n")
            return True
        except Exception as e:
            print(f"JSONL 파일 저장 중 오류 발생: {e}")
            print("\nStage 3: LLM 기반 초벌 JSON 생성 완료 (저장 실패).\n")
            return False
    else:
        print("저장할 유효한 결과 데이터가 없습니다.")
        print("\nStage 3: LLM 기반 초벌 JSON 생성 완료 (데이터 없음).\n")
        return False


# === Stage 4: Fine-tuning용 JSONL 파일 준비 (검토 단계 생략) ===
# **경고:** 이 단계는 Human Review를 생략합니다. 실제 프로젝트에서는 반드시 검토/수정 필요!
# ... (이전 답변에서 제공한 prepare_final_dataset 함수 그대로 사용)
def prepare_final_dataset(pre_review_path, final_path):
    """초벌 데이터를 최종 튜닝 데이터로 복사합니다 (Human Review 생략 시)."""
    print("\nStage 4: Fine-tuning용 최종 데이터셋 준비 시작 (Human Review 생략)")
    try:
        if os.path.exists(pre_review_path):
             shutil.copyfile(pre_review_path, final_path)
             print(f"초벌 데이터를 최종 튜닝 파일 '{final_path}'로 복사했습니다.")
             print("경고: Human Review 단계가 생략되었습니다. 데이터 품질을 보장할 수 없습니다.")
             print("Stage 4: 최종 데이터셋 준비 완료.\n")
             return True
        else:
             print(f"오류: 초벌 데이터 파일 '{pre_review_path}'를 찾을 수 없습니다.")
             print("Stage 4: 최종 데이터셋 준비 완료 (파일 없음).\n")
             return False
    except Exception as e:
        print(f"최종 데이터셋 준비 중 오류 발생: {e}")
        print("Stage 4: 최종 데이터셋 준비 완료 (오류).\n")
        return False

# === Stage 5: Google AI Studio API로 Fine-tuning 작업 시작 ===
# ... (이전 답변에서 제공한 start_fine_tuning_job 함수 그대로 사용)

# === Stage 5: Google AI Studio API로 Fine-tuning 작업 시작 ===

def start_fine_tuning_job(dataset_path, base_model_id, tuned_model_display_name, api_key):
    """Google AI Studio API를 사용하여 Fine-tuning 작업을 시작합니다."""
    print("Stage 5: Google AI Studio Fine-tuning 작업 시작")
    try:
        # 1. 데이터셋 파일 업로드 (MIME 타입 지정)
        print(f"데이터셋 파일 업로드 중: {dataset_path}")
        unique_file_name = f"snatch-tuning-data-{uuid.uuid4()}.jsonl"
        tuning_file = genai.upload_file(
            path=dataset_path,
            display_name=unique_file_name,
            mime_type="application/jsonl" # MIME 타입 명시
        )
        print(f"파일 업로드 요청 완료. File Name: {tuning_file.name}")

        # 파일 처리가 완료될 때까지 대기
        print("파일 처리 완료 대기 중...")
        file_status_model = genai.get_file(tuning_file.name)
        # 파일 상태가 ACTIVE 또는 FAILED가 될 때까지 대기
        while file_status_model.state.name not in ["ACTIVE", "FAILED"]:
            print(f"  파일 처리 중... (상태: {file_status_model.state.name})")
            time.sleep(10)
            file_status_model = genai.get_file(tuning_file.name)

        if file_status_model.state.name == "FAILED":
            # 파일 처리 실패 시 상세 정보 포함하여 예외 발생
            raise Exception(f"파일 처리 실패: {tuning_file.name}. 상태 정보: {file_status_model}")
        elif file_status_model.state.name == "ACTIVE":
             print(f"파일 처리 완료. 상태: {file_status_model.state.name}")
             print(f"File URI: {file_status_model.uri}")
        else:
            # 예상치 못한 상태 처리
            raise Exception(f"파일 처리 중 예상치 못한 상태 발생: {file_status_model.state.name}")


        # 2. Fine-tuning 작업 생성 (CreateTunedModel 사용)

        # --- 기반 모델 ID 확인 ('-tuning' 접미사) ---
        # Google AI Studio / Vertex AI 문서에 따르면, base_model은 'models/' 접두사가 필요 없을 수 있음.
        # create_tuned_model의 source_model 인자는 'models/...' 형식을 기대할 수 있음. 확인 필요.
        # 여기서는 전달받은 base_model_id를 그대로 사용하되, 'models/' 접두사 추가 및 접미사 확인
        tuning_base_model_name = base_model_id # 원본 ID 보존
        if not base_model_id.endswith('-tuning'):
            print(f"경고: 기반 모델 ID '{base_model_id}'에 '-tuning' 접미사가 없습니다. Fine-tuning API는 접미사가 있는 모델 ID를 요구할 수 있습니다 (예: gemini-1.5-flash-001-tuning).")
            # 만약 접미사가 반드시 필요하다면, 여기서 모델 ID를 강제로 수정하거나 에러 처리 필요.
            # 예시: base_model_id += '-tuning' # 강제 추가 (API 동작 보장 없음)
            # 또는 예외 발생: raise ValueError("Fine-tuning에는 '-tuning' 접미사가 있는 모델 ID가 필요합니다.")

        source_model_full_id = f'models/{base_model_id}' # API 호출 시 사용할 ID (접두사 포함)
        print(f"Fine-tuning 작업 생성 시작 (Base Model: {source_model_full_id})")
        # -----------------------------------------------

        # --- training_data 구조 수정 ---
        training_data_payload = {
            "examples": { # 데이터셋 위치 지정
                "dataset": {
                    "file_uri": file_status_model.uri
                }
            },
            # input_key와 output_key를 examples와 동일한 레벨로 이동
            "input_key": "textInput",
            "output_key": "output"
        }
        # ------------------------------

        # --- Fine-tuning 작업 시작 요청 ---
        operation = genai.create_tuned_model(
            # source_model: API 요구사항에 따라 'models/' 접두사 포함/미포함 확인 필요
            source_model=source_model_full_id, # '-tuning' 접미사 확인된 ID 사용
            training_data=training_data_payload, # 수정된 페이로드 전달
            # id: 튜닝된 모델을 식별하는 고유 ID (하이픈, 숫자, 소문자만 가능, 63자 이하)
            #     충돌 방지를 위해 고유하게 생성하는 것이 좋음.
            id=f"snatch-{tuned_model_display_name.lower().replace(' ', '-')}-{datetime.now().strftime('%y%m%d%H%M')}",
            display_name=tuned_model_display_name, # AI Studio/Vertex AI에 표시될 이름
            description="SNATCH 프로젝트용 리뷰 분석 모델 (편의점 디저트 VoC)",
            # 하이퍼파라미터 설정 (필요시 주석 해제 및 값 조정)
            # 참고: 사용 가능한 하이퍼파라미터와 기본값은 모델 및 API 버전에 따라 다름
            # hyperparameters={
            #     'epoch_count': 4,       # 전체 데이터셋 학습 반복 횟수
            #     'batch_size': 16,       # 한 번의 학습 단계에서 사용할 데이터 샘플 수
            #     'learning_rate': 0.001  # 모델 가중치 업데이트 강도
            # }
        )
        # ------------------------------

        print("Fine-tuning 작업 생성 요청 완료.")
        # operation 객체는 LRO(Long-Running Operation) 정보를 포함
        print("작업 ID (Operation Name):", operation.operation.name) # LRO 이름 출력
        print("\n=== 다음 단계 ===")
        print(f"Google AI Studio 웹 UI 또는 Vertex AI 콘솔에서 '{operation.operation.name}' 작업 진행 상황을 모니터링하세요.")
        print("튜닝이 완료되면 결과 모델 ID가 생성됩니다.")
        print("Stage 5: Fine-tuning 작업 시작 완료.\n")
        return operation.operation.name # 장기 실행 작업 이름 반환

    except Exception as e:
        print(f"Fine-tuning 작업 시작 중 오류 발생: {e}")
        # 오류 세부 정보 추가 출력 (디버깅용)
        # google.api_core.exceptions.GoogleAPICallError 같은 특정 오류 타입 확인 가능
        if isinstance(e, core_exceptions.GoogleAPICallError) and hasattr(e, 'response') and hasattr(e.response, 'text'):
            print(f"오류 응답 내용: {e.response.text}")
        elif hasattr(e, 'args') and e.args:
             print(f"오류 상세 정보: {e.args}")

        print("\nStage 5: Fine-tuning 작업 시작 완료 (오류).\n")
        return None

# === 메인 실행 로직 ===
if __name__ == "__main__" and settings_ok:
    # Stage 1: 데이터 수집
    df_raw = collect_youtube_data(API_KEY, SEARCH_QUERY, NUM_VIDEOS_TO_SEARCH, NUM_COMMENTS_PER_VIDEO, RAW_COMMENTS_CSV)

    if df_raw is not None and not df_raw.empty:
        # Stage 2: 데이터 전처리
        df_filtered = preprocess_and_filter_comments(df_raw)

        if not df_filtered.empty:
            # Stage 3: LLM 초벌 생성
            # GOOGLE_API_KEY 환경 변수 또는 configure 방식으로 API 키 로드 필요 (SDK 버전 확인)
            gen_success = generate_llm_outputs(df_filtered, GENERATION_MODEL_ID, PRE_REVIEW_JSONL_PATH, MAX_COMMENTS_TO_PROCESS, API_CALL_DELAY, GOOGLE_API_KEY)

            if gen_success:
                 # Stage 4: 최종 데이터셋 준비 (Human Review 생략)
                 prep_success = prepare_final_dataset(PRE_REVIEW_JSONL_PATH, FINAL_TUNING_JSONL_PATH)

                 if prep_success:
                    # Stage 5: Fine-tuning 시작
                    # GOOGLE_API_KEY 환경 변수 또는 configure 방식으로 API 키 로드 필요 (SDK 버전 확인)
                    tuning_job_name = start_fine_tuning_job(FINAL_TUNING_JSONL_PATH, TUNING_BASE_MODEL_ID, TUNED_MODEL_DISPLAY_NAME, GOOGLE_API_KEY)
                    if tuning_job_name:
                        print("=== 전체 프로세스 완료 (Fine-tuning 작업 시작됨) ===")
                    else:
                        print("오류: Fine-tuning 작업을 시작하지 못했습니다.")
                 else:
                     print("오류: 최종 데이터셋 준비에 실패했습니다.")
            else:
                 print("오류: LLM 기반 초벌 데이터 생성에 실패했습니다.")
        else:
            print("정보: 필터링 후 처리할 유효한 댓글이 없습니다.")
    else:
        print("정보: YouTube에서 댓글을 수집하지 못했거나 데이터가 없습니다.")


Stage 1: YouTube 댓글 데이터 수집 시작
YouTube 클라이언트 초기화 성공.
'CU 밤티라미수' 검색 결과: 3개의 영상을 찾았습니다 (최대 3개).


영상별 댓글 수집 중:   0%|          | 0/3 [00:00<?, ?it/s]


영상 처리 중: CU밤티라미수컵🌰 (ID: jWrZjXaozIg)
  - 영상 ID 'jWrZjXaozIg': 4개의 댓글 수집 완료.

영상 처리 중: 나폴리 맛피아님 밤티라미수 개선판이 나왔습니다 (ID: VOL3vIPk3Ec)
  - 영상 ID 'VOL3vIPk3Ec': 50개의 댓글 수집 완료.

영상 처리 중: 나폴리 맛피아 CU 밤티라미수 후기 죄송하지만 보류입니다.. (ID: tgntgIgXvG0)
  - 영상 ID 'tgntgIgXvG0': 30개의 댓글 수집 완료.

총 84개의 댓글을 수집했습니다.
Raw 댓글 데이터를 'CU_밤티라미수_raw_comments.csv'로 저장했습니다.
Stage 1: YouTube 댓글 데이터 수집 완료.

Stage 2: 데이터 전처리 및 필터링 시작
초기 댓글 수 = 84
  - 길이 필터링 후: 77 (제외: 7)


  - 언어 감지 중:   0%|          | 0/77 [00:00<?, ?it/s]

  - 한국어 필터링 후: 77 (제외: 0)
  - 규칙 기반 필터링 후: 77 (제외: 0)
Stage 2: 데이터 전처리 및 필터링 완료. 최종 댓글 수 = 77


Stage 3: LLM 기반 초벌 JSON 생성 시작
총 10개의 댓글에 대해 LLM 분석을 시도합니다 (최대 10개).


<ipython-input-3-1742e37c0cc0>:228: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  noise_mask = ~df_processed['comment_text_clean'].str.contains(combined_pattern, regex=True, na=False)


LLM 분석 중:   0%|          | 0/10 [00:00<?, ?it/s]


Stage 3: LLM 기반 초벌 JSON 생성 완료. 성공 10건, 실패 0건

결과를 JSONL 파일로 저장 중: generated_prereview_data.jsonl
JSONL 파일 저장 완료.

Stage 3: LLM 기반 초벌 JSON 생성 완료.


Stage 4: Fine-tuning용 최종 데이터셋 준비 시작 (Human Review 생략)
초벌 데이터를 최종 튜닝 파일 'final_finetuning_data.jsonl'로 복사했습니다.
경고: Human Review 단계가 생략되었습니다. 데이터 품질을 보장할 수 없습니다.
Stage 4: 최종 데이터셋 준비 완료.

Stage 5: Google AI Studio Fine-tuning 작업 시작
데이터셋 파일 업로드 중: final_finetuning_data.jsonl
파일 업로드 요청 완료. File Name: files/yrs9229gds7p
파일 처리 완료 대기 중...
파일 처리 완료. 상태: ACTIVE
File URI: https://generativelanguage.googleapis.com/v1beta/files/yrs9229gds7p
Fine-tuning 작업 생성 시작 (Base Model: models/gemini-1.5-flash-001-tuning)
Fine-tuning 작업 시작 중 오류 발생: "Invalid key: The input key 'text_input' does not exist in the data. Available keys are: ['examples', 'input_key', 'output_key']."
오류 상세 정보: ("Invalid key: The input key 'text_input' does not exist in the data. Available keys are: ['examples', 'input_key', 'output_key'].",)

Stage 5: Fine-tuning 작업 시작 완료 (오류).

오류: Fine-t

In [5]:
# === 메인 실행 로직 ===
if __name__ == "__main__" and settings_ok:
    # Stage 1: 데이터 수집
    df_raw = collect_youtube_data(API_KEY, SEARCH_QUERY, NUM_VIDEOS_TO_SEARCH, NUM_COMMENTS_PER_VIDEO, RAW_COMMENTS_CSV)

    if df_raw is not None and not df_raw.empty:
        # Stage 2: 데이터 전처리
        df_filtered = preprocess_and_filter_comments(df_raw)

        if not df_filtered.empty:
            # Stage 3: LLM 초벌 생성
            # GOOGLE_API_KEY 환경 변수 또는 configure 방식으로 API 키 로드 필요 (SDK 버전 확인)
            gen_success = generate_llm_outputs(df_filtered, GENERATION_MODEL_ID, PRE_REVIEW_JSONL_PATH, MAX_COMMENTS_TO_PROCESS, API_CALL_DELAY, GOOGLE_API_KEY)

            if gen_success:
                 # Stage 4: 최종 데이터셋 준비 (Human Review 생략)
                 prep_success = prepare_final_dataset(PRE_REVIEW_JSONL_PATH, FINAL_TUNING_JSONL_PATH)

                 if prep_success:
                    # Stage 5: Fine-tuning 시작
                    # GOOGLE_API_KEY 환경 변수 또는 configure 방식으로 API 키 로드 필요 (SDK 버전 확인)
                    tuning_job_name = start_fine_tuning_job(FINAL_TUNING_JSONL_PATH, TUNING_BASE_MODEL_ID, TUNED_MODEL_DISPLAY_NAME, GOOGLE_API_KEY)
                    if tuning_job_name:
                        print("=== 전체 프로세스 완료 (Fine-tuning 작업 시작됨) ===")
                    else:
                        print("오류: Fine-tuning 작업을 시작하지 못했습니다.")
                 else:
                     print("오류: 최종 데이터셋 준비에 실패했습니다.")
            else:
                 print("오류: LLM 기반 초벌 데이터 생성에 실패했습니다.")
        else:
            print("정보: 필터링 후 처리할 유효한 댓글이 없습니다.")
    else:
        print("정보: YouTube에서 댓글을 수집하지 못했거나 데이터가 없습니다.")


Stage 1: YouTube 댓글 데이터 수집 시작
YouTube 클라이언트 초기화 성공.
'CU 밤티라미수' 검색 결과: 3개의 영상을 찾았습니다 (최대 3개).


영상별 댓글 수집 중:   0%|          | 0/3 [00:00<?, ?it/s]


영상 처리 중: CU밤티라미수컵🌰 (ID: jWrZjXaozIg)
  - 영상 ID 'jWrZjXaozIg': 4개의 댓글 수집 완료.

영상 처리 중: 나폴리 맛피아님 밤티라미수 개선판이 나왔습니다 (ID: VOL3vIPk3Ec)
  - 영상 ID 'VOL3vIPk3Ec': 50개의 댓글 수집 완료.

영상 처리 중: 나폴리 맛피아 CU 밤티라미수 후기 죄송하지만 보류입니다.. (ID: tgntgIgXvG0)
  - 영상 ID 'tgntgIgXvG0': 30개의 댓글 수집 완료.

총 84개의 댓글을 수집했습니다.
Raw 댓글 데이터를 'CU_밤티라미수_raw_comments.csv'로 저장했습니다.
Stage 1: YouTube 댓글 데이터 수집 완료.

Stage 2: 데이터 전처리 및 필터링 시작
초기 댓글 수 = 84
  - 길이 필터링 후: 77 (제외: 7)


  - 언어 감지 중:   0%|          | 0/77 [00:00<?, ?it/s]

  - 한국어 필터링 후: 77 (제외: 0)
  - 규칙 기반 필터링 후: 77 (제외: 0)
Stage 2: 데이터 전처리 및 필터링 완료. 최종 댓글 수 = 77


Stage 3: LLM 기반 초벌 JSON 생성 시작
총 10개의 댓글에 대해 LLM 분석을 시도합니다 (최대 10개).


<ipython-input-3-1742e37c0cc0>:228: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  noise_mask = ~df_processed['comment_text_clean'].str.contains(combined_pattern, regex=True, na=False)


LLM 분석 중:   0%|          | 0/10 [00:00<?, ?it/s]


[Row 1/10] 처리 시작. 댓글: #cu#흑백요리사#밤티라미수컵#밤티라미수
@cu_official 
🌰흑백요리사에 나온 밤티...
[Row 1] LLM API 호출 시도...
[Row 1] LLM API 호출 완료.
[Row 1] 결과 저장됨.
[Row 1] API 호출 간 지연 (1.5초)...

[Row 2/10] 처리 시작. 댓글: ❤공들여 만든 소중한 영상이니
악플은 자제해주세요!!! 
영상이 본인과 무관하다 생각드시면...
[Row 2] LLM API 호출 시도...
[Row 2] LLM API 호출 완료.
[Row 2] 결과 저장됨.
[Row 2] API 호출 간 지연 (1.5초)...

[Row 3/10] 처리 시작. 댓글: 얌얌디너님의 자세한설명 좋아요달콤 고소하겠어요❤...
[Row 3] LLM API 호출 시도...
[Row 3] LLM API 호출 완료.
[Row 3] 결과 저장됨.
[Row 3] API 호출 간 지연 (1.5초)...

[Row 4/10] 처리 시작. 댓글: 이거 개선 됀거 드신건가용?...
[Row 4] LLM API 호출 시도...
[Row 4] LLM API 호출 완료.
[Row 4] 결과 저장됨.
[Row 4] API 호출 간 지연 (1.5초)...

[Row 5/10] 처리 시작. 댓글: 롱폼 뭐야 숏츠 3분 된다며 유튜브 놈들아...
[Row 5] LLM API 호출 시도...
[Row 5] LLM API 호출 완료.
[Row 5] 결과 저장됨.
[Row 5] API 호출 간 지연 (1.5초)...

[Row 6/10] 처리 시작. 댓글: 00:59 ??? : 이야~~료이키텐카이...
[Row 6] LLM API 호출 시도...
[Row 6] LLM API 호출 완료.
[Row 6] 결과 저장됨.
[Row 6] API 호출 간 지연 (1.5초)...

[Row 7/10] 처리 시작. 댓글: 나폴리 맛피자 나폴리 맛조개 ㅇㅈ랄 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ...
[Row 7] LLM API